<h2>Camshift </h2>
<p>Did you closely watch the last result? There is a problem. Our window always has the same size when the object is farther away and it is very close to camera. That is not good. We need to adapt the window size with size and rotation of the target. Once again, the solution came from "OpenCV Labs" and it is called CAMshift (Continuously Adaptive Meanshift) published by Gary Bradsky in his paper "Computer Vision Face Tracking for Use in a Perceptual User Interface" in 1988.</p>
<p>It applies meanshift first. Once meanshift converges, it updates the size of the window as, \(s = 2 \times \sqrt{\frac{M_{00}}{256}}\). It also calculates the orientation of best fitting ellipse to it. Again it applies the meanshift with new scaled search window and previous window location. The process is continued until required accuracy is met.</p>
<div class="image">
<img src="https://docs.opencv.org/4.0.0/camshift_face.gif" alt="camshift_face.gif"/>
<div class="caption">
image</div></div>
 <h3>Camshift in OpenCV.js</h3>
<p>It is almost same as meanshift, but it returns a rotated rectangle (that is our result) and box parameters (used to be passed as search window in next iteration).</p>
<p>We use the function: <b><a class="el" href="../../dc/d6b/group__video__track.html#gaef2bd39c8356f423124f1fe7c44d54a1" title="Finds an object center, size, and orientation. ">cv.CamShift</a> (probImage, window, criteria)</b> </p><dl class="params"><dt>Parameters</dt><dd>
  <table class="params">
    <tr><td class="paramname">probImage</td><td>Back projection of the object histogram. See <a class="el" href="../../d6/dc7/group__imgproc__hist.html#ga3a0af640716b456c3d14af8aee12e3ca" title="Calculates the back projection of a histogram. ">cv.calcBackProject</a> for details. </td></tr>
    <tr><td class="paramname">window</td><td>Initial search window. </td></tr>
    <tr><td class="paramname">criteria</td><td>Stop criteria for the iterative search algorithm. </td></tr>
  </table>
  </dd>
</dl>
<dl class="section return"><dt>Returns</dt><dd>Rotated rectangle and the new search window</dd></dl>

<h2>Additional Resources </h2>
<ol type="1">
<li>French Wikipedia page on <a href="http://fr.wikipedia.org/wiki/Camshift">Camshift</a>. (The two animations are taken from here)</li>

In [3]:
import numpy as np
import cv2 as cv

path = '/Users/its_agni/Documents/All programming files/Computer_Vision/data/' 
cap = cv.VideoCapture(path + 'slow_traffic_small.mp4')
# take first frame of the video
ret, frame = cap.read()

# setup initial location of window
x, y, width, height = 300, 200, 100, 50
track_window = (x, y ,width, height)

# set up the ROI for tracking
roi = frame[y:y+height, x : x+width]
hsv_roi = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
mask = cv.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255)))
roi_hist = cv.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv.normalize(roi_hist, roi_hist, 0, 255,cv.NORM_MINMAX)

# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1)
cv.imshow('roi',roi)
while(1):
    ret, frame = cap.read()
    if ret == True:

        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        # apply meanshift to get the new location
        ret, track_window = cv.CamShift(dst, track_window, term_crit)
        # camshift will return (x,y,width,height,rotataion)
        
        # Draw it on image
        pts = cv.boxPoints(ret)
        print(pts)
        pts = np.int0(pts)
        final_image = cv.polylines(frame, [pts], True, (0, 255, 0), 2)
        #x,y,w,h = track_window
        #final_image = cv.rectangle(frame, (x,y), (x+w, y+h), 255, 3)

        cv.imshow('dst', dst)
        cv.imshow('final_image',final_image)
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break

[[315.2072  187.89851]
 [398.84894 194.02357]
 [396.7928  222.10149]
 [313.15106 215.97643]]
[[319.61652 191.50002]
 [394.9226  196.85487]
 [393.38348 218.49998]
 [318.0774  213.14513]]
[[321.63092 192.25986]
 [394.95935 198.23984]
 [393.36908 217.74014]
 [320.04065 211.76016]]
[[320.7584  193.52069]
 [393.6008  198.82953]
 [392.2416  217.47931]
 [319.3992  212.17047]]
[[321.82224 194.5947 ]
 [393.54575 199.8956 ]
 [392.17776 218.4053 ]
 [320.45425 213.1044 ]]
[[320.9231  193.14307]
 [399.94702 200.96628]
 [398.0769  219.85693]
 [319.05298 212.03372]]
[[321.89383 193.63565]
 [400.0682  200.7232 ]
 [398.10617 222.36435]
 [319.9318  215.2768 ]]
[[321.41946 192.9755 ]
 [401.00034 200.98657]
 [398.58054 225.0245 ]
 [318.99966 217.01343]]
[[323.20358 194.14804]
 [403.91214 201.94138]
 [401.79642 223.85196]
 [321.08786 216.05862]]
[[321.38187 193.90353]
 [404.7753  197.7038 ]
 [403.61813 223.09647]
 [320.2247  219.2962 ]]
[[317.10464 194.3696 ]
 [406.2131  195.4097 ]
 [405.89536 222.6304 ]
 

[[344.81067 199.05914]
 [522.21594 193.10864]
 [524.18933 251.94086]
 [346.78406 257.89136]]
[[342.88327 198.88121]
 [522.0847  192.65297]
 [524.1167  251.11879]
 [344.91528 257.34705]]
[[347.2617  200.74075]
 [516.74927 191.34427]
 [519.7383  245.25925]
 [350.25073 254.65573]]
[[340.9721  203.49875]
 [520.9445  190.20792]
 [525.0279  245.50125]
 [345.05548 258.79208]]
[[344.5982  200.86084]
 [524.9692  192.78809]
 [527.4018  247.13916]
 [347.03082 255.21191]]
[[343.74216 201.1772 ]
 [530.54083 192.34685]
 [533.2578  249.8228 ]
 [346.45917 258.65314]]
[[349.54086 201.17953]
 [530.07574 190.36075]
 [533.4591  246.82047]
 [352.92426 257.63925]]
[[346.93326 202.92288]
 [528.31744 190.83197]
 [532.0668  247.07712]
 [350.68256 259.16803]]
[[352.45227 201.37921]
 [532.6931  192.2515 ]
 [535.5477  248.62079]
 [355.3069  257.7485 ]]
[[350.19757 202.90611]
 [535.64264 192.84633]
 [538.8024  251.09389]
 [353.35736 261.1537 ]]
[[352.7795  200.30261]
 [534.46356 191.67062]
 [537.22046 249.69739]
 

[[554.3558  357.4738 ]
 [599.9191  198.03036]
 [657.6442  214.52621]
 [612.0809  373.96964]]
[[564.82996 364.8094 ]
 [598.9994  205.56696]
 [653.17004 217.19061]
 [619.0006  376.43304]]
[[571.45135 366.88153]
 [599.9996  214.87376]
 [654.54865 225.11847]
 [626.0004  377.12622]]
[[579.70557 370.74115]
 [597.5679  226.2464 ]
 [654.29443 233.25885]
 [636.4321  377.7536 ]]
[[587.31714 365.3786 ]
 [599.34424 243.55305]
 [650.68286 248.6214 ]
 [638.65576 370.44696]]
[[594.44745 362.25928]
 [602.6932  256.25134]
 [647.55255 259.74072]
 [639.3068  365.74866]]
[[644.4025  273.45255]
 [647.03326 359.27737]
 [605.5975  360.54745]
 [602.96674 274.72263]]
[[642.531   283.52274]
 [648.0348  352.5623 ]
 [611.469   355.47726]
 [605.9652  286.4377 ]]
[[640.484   289.63007]
 [645.52185 348.99323]
 [617.516   351.36993]
 [612.47815 292.00677]]
[[620.39264 343.7222 ]
 [622.71954 288.39926]
 [643.60736 289.2778 ]
 [641.28046 344.60074]]
[[624.19    343.0985 ]
 [630.6064  294.16998]
 [643.81    295.9015 ]
 

[[518.7324   -38.59076 ]
 [682.49866   69.072205]
 [603.2676   189.59076 ]
 [439.50134   81.927795]]
[[509.86377  -28.824528]
 [678.0252    62.502655]
 [612.1362   183.82452 ]
 [443.9748    92.497345]]
[[508.3496   -27.67657 ]
 [677.1406    63.691166]
 [611.6504   184.67657 ]
 [442.85938   93.30884 ]]
[[506.64346  -28.8988  ]
 [676.71466   63.176   ]
 [611.35657  183.8988  ]
 [441.28534   91.824005]]
[[508.3357   -27.400566]
 [678.08826   67.16744 ]
 [611.6643   186.40057 ]
 [441.91174   91.83256 ]]
[[505.18414  -31.429688]
 [675.1268    57.41382 ]
 [610.81586  180.42969 ]
 [440.87323   91.58618 ]]
[[505.09988  -36.042477]
 [678.51      60.114765]
 [610.90015  182.04248 ]
 [437.49      85.88524 ]]
[[502.35715 -38.69408]
 [675.1551   54.45744]
 [607.6428  179.69408]
 [434.8449   86.54256]]
[[499.91052  -39.05381 ]
 [677.02136   54.953876]
 [610.0895   181.0538  ]
 [432.97864   87.04613 ]]
[[489.5921   -41.390255]
 [674.72626   47.288208]
 [612.4079   177.39026 ]
 [427.27374   88.71179 ]

[[515.494    -44.996235]
 [685.3613    59.27263 ]
 [604.506    190.99623 ]
 [434.63867   86.72737 ]]
[[512.35846  -43.671116]
 [679.8272    53.877197]
 [603.64154  184.67111 ]
 [436.1728    87.1228  ]]
[[508.40094  -42.06969 ]
 [680.63385   53.637123]
 [607.59906  185.06969 ]
 [435.36615   89.36288 ]]
[[508.7859   -42.962883]
 [680.182     51.33207 ]
 [607.2141   183.96289 ]
 [435.818     89.66793 ]]
[[515.23334 -42.37755]
 [682.498    58.53044]
 [604.76666 187.37755]
 [437.502    86.46956]]
[[507.80038  -46.60703 ]
 [680.9559    60.3847  ]
 [604.1996   184.60703 ]
 [431.04413   77.615295]]
[[492.25803 -42.65189]
 [674.8458   44.60495]
 [611.74194 176.65189]
 [429.15417  89.39505]]
[[490.00684  -43.13973 ]
 [676.67004   44.702744]
 [611.99316  182.13972 ]
 [425.32996   94.29726 ]]
[[490.10876 -44.37402]
 [683.0762   50.17936]
 [613.89124 191.37402]
 [420.92383  96.82064]]
[[496.6753   -49.333626]
 [683.2997    55.429527]
 [605.3247   194.33362 ]
 [418.70032   89.57047 ]]
[[492.82895  -

...

[[444.44556    285.41434   ]
 [445.62097      0.70240617]
 [659.55444      1.5856628 ]
 [658.379      286.2976    ]]
[[652.5545     -3.2350564]
 [663.90985   281.7699   ]
 [451.4455    290.23505  ]
 [440.09015     5.2301025]]
[[4.4646515e+02 2.8831381e+02]
 [4.4860699e+02 1.3338506e-01]
 [6.5753485e+02 1.6861877e+00]
 [6.5539301e+02 2.8986661e+02]]
[[651.77997   -2.579169]
 [661.892    287.1925  ]
 [450.22003  294.57916 ]
 [440.10797    4.807495]]
[[ 6.5426440e+02 -4.4465804e-01]
 [ 6.5832031e+02  2.9045273e+02]
 [ 4.4373560e+02  2.9344467e+02]
 [ 4.3967969e+02  2.5472717e+00]]
[[428.3422   288.57516 ]
 [445.54752   -5.454982]
 [665.65784    7.424835]
 [648.4525   301.455   ]]
[[423.18604   287.44598  ]
 [442.76257    -6.5007944]
 [668.81396     8.554016 ]
 [649.2374    302.5008   ]]
[[419.22958  287.35825 ]
 [447.88644  -10.028578]
 [672.7704    11.641754]
 [644.1135   309.02856 ]]
[[418.49423  287.9883  ]
 [450.05002  -10.79928 ]
 [675.50574   13.011688]
 [643.94995  311.7993  ]]
[[4

[[637.0038    -5.568899]
 [662.7346   343.77216 ]
 [488.99622  356.5689  ]
 [463.26538    7.227844]]
[[636.5226     -5.766796 ]
 [662.9177    340.70605  ]
 [491.47742   353.76678  ]
 [465.08228     7.2939453]]
[[636.21735   -5.762146]
 [664.2669   342.01184 ]
 [493.78265  355.76215 ]
 [465.7331     7.988159]]
[[632.7945     -6.5016527]
 [664.41034   341.30252  ]
 [497.2055    356.50165  ]
 [465.58966     8.697479 ]]
[[633.7255     -5.8256216]
 [662.9173    345.99887  ]
 [496.27448   359.82562  ]
 [467.0827      8.001129 ]]
[[636.0365     -4.7873445]
 [661.0763    346.7412   ]
 [491.9635    358.78735  ]
 [466.9237      7.258789 ]]
[[633.4368     -6.908948 ]
 [665.77545   340.14148  ]
 [496.56317   355.90894  ]
 [464.22455     8.8585205]]
[[635.11664    -5.7171144]
 [663.6409    340.99796  ]
 [496.88336   354.7171   ]
 [468.35907     8.002045 ]]
[[634.2407     -6.5821643]
 [666.74695   341.15942  ]
 [501.75928   356.58215  ]
 [469.25305     8.840576 ]]
[[633.27405    -4.1339808]
 [655.54

[[592.7547   -35.99643 ]
 [681.28656  137.4913  ]
 [537.2453   210.99643 ]
 [448.71344   37.508698]]
[[584.59015  -41.629303]
 [683.8222   120.07926 ]
 [544.40985  205.6293  ]
 [445.1778    43.92074 ]]
[[599.51666  -30.057058]
 [676.0568   146.11389 ]
 [533.48334  208.05705 ]
 [456.94318   31.886108]]
[[599.4079   -31.555805]
 [676.5367   144.18864 ]
 [527.5921   209.5558  ]
 [450.46332   33.811356]]
[[582.7507   -40.296143]
 [680.8545   127.51019 ]
 [539.2493   210.29614 ]
 [441.1455    42.489807]]
[[574.76196  -40.708675]
 [679.4862   126.00775 ]
 [546.23804  209.70868 ]
 [441.5138    42.99225 ]]
[[600.79755  -26.47235 ]
 [670.92035  152.26569 ]
 [530.20245  207.47235 ]
 [460.07965   28.734314]]
[[588.27075  -36.054726]
 [678.1587   137.12936 ]
 [535.72925  211.05472 ]
 [445.8413    37.870636]]
[[586.6172   -37.198788]
 [679.8325   132.68085 ]
 [540.3828   209.19879 ]
 [447.16748   39.319153]]
[[587.05646  -36.35183 ]
 [680.7863   135.0803  ]
 [544.94354  209.35184 ]
 [451.21368   37

[[588.62134  -32.750984]
 [679.1588   125.2688  ]
 [561.37866  192.75098 ]
 [470.8412    34.7312  ]]
[[576.0919   -36.054222]
 [682.21606  114.69691 ]
 [570.9081   193.05423 ]
 [464.78394   42.303093]]
[[579.8632   -32.781887]
 [678.48663  118.5755  ]
 [566.1368   191.78189 ]
 [467.51337   40.4245  ]]
[[578.6836   -33.35254 ]
 [681.1575   115.205986]
 [569.3164   192.35254 ]
 [466.84253   43.794014]]
[[572.63727  -34.09734 ]
 [678.9108   112.67084 ]
 [573.36273  189.09734 ]
 [467.08917   42.329163]]
[[593.87427  -27.60326 ]
 [676.28674  126.58504 ]
 [562.12573  187.60326 ]
 [479.71326   33.414963]]
[[580.1405   -31.114723]
 [679.76636  118.685265]
 [570.8595   191.11472 ]
 [471.23364   41.314735]]
[[595.21747  -27.876116]
 [676.69855  127.68419 ]
 [561.78253  187.87611 ]
 [480.30145   32.31581 ]]
[[580.1422  -33.51494]
 [679.5316  117.83883]
 [568.8578  190.51494]
 [469.46838  39.16117]]
[[571.8667   -34.945312]
 [672.49927  108.541756]
 [562.1333   185.94531 ]
 [461.50073   42.458244]

[[622.5231   -10.352772]
 [672.37854  283.82977 ]
 [539.4769   306.35278 ]
 [489.62146   12.170227]]
[[621.71277  -10.283106]
 [672.6465   286.57333 ]
 [540.28723  309.2831  ]
 [489.35352   12.426666]]
[[618.7396   -11.504596]
 [674.3197   282.74948 ]
 [543.2604   307.5046  ]
 [487.6803    13.250519]]
[[618.8753   -11.457805]
 [674.7758   281.32162 ]
 [543.1247   306.4578  ]
 [487.22418   13.678375]]
[[614.0673   -12.910446]
 [677.2857   279.36423 ]
 [549.9327   306.91046 ]
 [486.7143    14.635773]]
[[616.06714  -11.846161]
 [675.9304   278.90427 ]
 [549.93286  304.84616 ]
 [490.06958   14.095734]]
[[618.275    -10.4754  ]
 [672.92004  282.09177 ]
 [547.725    305.4754  ]
 [493.07996   12.908234]]
[[612.9241   -11.435172]
 [672.12646  280.47964 ]
 [549.0759   305.43518 ]
 [489.87354   13.520355]]
[[612.82104   -12.496812 ]
 [677.6969    283.17532  ]
 [553.17896   310.49683  ]
 [488.3031     14.8246765]]
[[617.01184  -11.50352 ]
 [674.61804  277.89432 ]
 [550.98816  302.5035  ]
 [493.38